In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
# put actual numpy arrays here please
# X.shape == (num_samples, seq_len=73, num_features)
# y.shape == (num_samples,)
X = np.random.randn(500, 73, 5)   # example: 500 county‑years, 5 features 
y = np.random.randn(500)         # example yield values

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class CropDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 32
dataset = CropDataset(X, y)
loader  = DataLoader(dataset, batch_size=batch_size, shuffle=True)

### LSTM Regressor

In [ ]:

class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=1, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim,
                            num_layers, batch_first=True, dropout=dropout)
        self.fc   = nn.Linear(hidden_dim, 1)
    def forward(self, x):
        # x: [B, T, F]
        _, (h_n, _) = self.lstm(x)        # h_n: [num_layers, B, hidden_dim]
        out = self.fc(h_n[-1])            # take last layer's hidden state
        return out.squeeze(1)             # [B]

# instantiate
input_dim = X.shape[-1]
lstm_model = LSTMRegressor(input_dim).to(device)

### TCN Regressor

In [ ]:
class TemporalBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, stride, dilation, padding, dropout=0.2):
        super().__init__()
        self.conv1 = nn.utils.weight_norm(
            nn.Conv1d(in_ch, out_ch, kernel_size,
                      stride=stride, padding=padding, dilation=dilation)
        )
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.utils.weight_norm(
            nn.Conv1d(out_ch, out_ch, kernel_size,
                      stride=stride, padding=padding, dilation=dilation)
        )
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.downsample = (
            nn.Conv1d(in_ch, out_ch, 1) if in_ch != out_ch else None
        )
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.dropout1(out)

        out = self.conv2(out)
        out = self.relu2(out)
        out = self.dropout2(out)

        res = x if self.downsample is None else self.downsample(x)
        return self.relu2(out + res)

class TCN(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=3, dropout=0.2):
        super().__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            in_ch = num_inputs if i == 0 else num_channels[i-1]
            out_ch = num_channels[i]
            dilation = 2 ** i
            padding  = (kernel_size-1) * dilation
            layers.append(
                TemporalBlock(in_ch, out_ch, kernel_size,
                              stride=1, dilation=dilation,
                              padding=padding, dropout=dropout)
            )
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        # x: [B, T, F] -> [B, F, T] for Conv1d
        x = x.transpose(1, 2)
        y = self.network(x)
        # y: [B, C_last, T] -> take last time step
        return y[:, :, -1]

class TCNRegressor(nn.Module):
    def __init__(self, input_dim, channel_sizes=[32, 32, 64], kernel_size=3):
        super().__init__()
        self.tcn = TCN(input_dim, channel_sizes, kernel_size=kernel_size)
        self.fc  = nn.Linear(channel_sizes[-1], 1)
    def forward(self, x):
        # x: [B, T, F]
        y = self.tcn(x)     # [B, C_last]
        return self.fc(y).squeeze(1)

# instantiate
tcn_model = TCNRegressor(input_dim).to(device)

### Training loop

In [ ]:
def train_model(model, loader, lr=1e-3, epochs=20):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    for ep in range(1, epochs+1):
        total_loss = 0
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            pred   = model(xb)
            loss   = criterion(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * xb.size(0)
        avg_loss = total_loss / len(loader.dataset)
        print(f"{model.__class__.__name__} | Epoch {ep:02d} | Loss: {avg_loss:.4f}")

# Train LSTM
train_model(lstm_model, loader, lr=1e-3, epochs=10)
# Train TCN
train_model(tcn_model, loader, lr=1e-3, epochs=10)

quick eval

In [ ]:
def evaluate(model):
    model.eval()
    with torch.no_grad():
        xb = torch.tensor(X, dtype=torch.float32).to(device)
        yb = torch.tensor(y, dtype=torch.float32).to(device)
        preds = model(xb)
        mse   = nn.MSELoss()(preds, yb).item()
    print(f"{model.__class__.__name__} final MSE: {mse:.4f}")

evaluate(lstm_model)
evaluate(tcn_model)